In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab - not in local mode
%cd /content/drive/My Drive/DMTM

/content/drive/My Drive/DMTM


In [84]:
# list content of drive
%ls

IG_Sample1.txt  IG_Sample2.txt  TestParameter.txt  TestTransaction.txt


In [99]:
# READ IG Sample File 
records = []

fileData = open('IG_Sample2.txt', "r+").readlines()
  
# remove all illegal whitespaces, all newline chars, trailing commas that have no input text after it
records = [line.replace(" ", "").rstrip("\n").rstrip(",") for line in fileData]

records = [list(map(str, line.split(","))) for line in records]

records

[['young', 'false', 'false', 'fair', 'No'],
 ['young', 'false', 'false', 'good', 'No'],
 ['young', 'true', 'false', 'good', 'Yes'],
 ['young', 'true', 'true', 'fair', 'Yes'],
 ['young', 'false', 'false', 'fair', 'No'],
 ['middle', 'false', 'false', 'fair', 'No'],
 ['middle', 'false', 'false', 'good', 'No'],
 ['middle', 'true', 'true', 'good', 'Yes'],
 ['middle', 'false', 'true', 'excellent', 'Yes'],
 ['middle', 'false', 'true', 'excellent', 'Yes'],
 ['old', 'false', 'true', 'excellent', 'Yes'],
 ['old', 'false', 'true', 'good', 'Yes'],
 ['old', 'true', 'false', 'good', 'Yes'],
 ['old', 'true', 'false', 'excellent', 'Yes'],
 ['old', 'false', 'false', 'fair', 'No']]

In [100]:
# get all classes distinct ==? 
# TODO: do we need to upper/lower case them ?
classes = list(set([line[-1] for line in records]))
classes

['Yes', 'No']

In [101]:
import math

# Calculate entropy wrt classes 
E_Y = float()

# loop thru classes
for idx, c in enumerate(classes):
  n_rows_class = 0

  for r in records:
    # number of rows having this class
    if r[-1] == c :
      n_rows_class += 1  

  # calc class entropy here 
  pr_c = n_rows_class / len(records)
  E_Y += ( -pr_c) * math.log((pr_c),2)

  print(c, n_rows_class) 
E_Y
  


Yes 9
No 6


0.9709505944546686

In [102]:
import math

def fn_attr_entropy(D_i):
  # Calculate entropy wrt classes 
  E_x = float()

  # loop thru classes
  for idx, c in enumerate(classes):
    n_rows_class = 0

    for r in D_i:
      # number of rows having this class
      if r[-1] == c :
        n_rows_class += 1  

    if n_rows_class == 0:
      continue

    # calc class entropy here 
    pr_c = n_rows_class / len(D_i)
    E_x += ( -pr_c) *  math.log((pr_c),2)

    #print(c, n_rows_class) 
  return E_x


In [103]:
def IG_ATTR_LOOPER(IDX, precision_req):
  # Calculate a dict of attribute value counts for records[i][IDX]
  attr_vals = {}

  for r in records:
    if r[IDX] in attr_vals.keys():
      attr_vals[r[IDX]] += 1
    else:
      attr_vals[r[IDX]] = 1

  # partial entropies 
  E_vi = float()

  #print(attr_vals)

  for val in attr_vals.keys():
    D_i = [r for r in records if r[IDX] == val]
    pr_c = attr_vals[val] / len(records)

    # calculate sum of partial entropies here 
    E_vi += pr_c * fn_attr_entropy(D_i)


  print('\nENTR for '+str(IDX)+' : ', format(E_vi, precision_req))
  #print('GAIN for '+str(IDX)+' : ', format(E_Y - E_vi, precision_req))
  return '(IG '+ format(E_Y - E_vi, precision_req) + ')'



In [104]:
# SPECIFY ATTRIBUTE COLUMN YOU WANT HERE 
precision_req =  '.3f'

print('TOTAL ENTR : ', E_Y)
for IDX in range(len(records[0])-1):
   print(IG_ATTR_LOOPER(IDX, precision_req))

TOTAL ENTR :  0.9709505944546686

ENTR for 0 :  0.888
(IG 0.083)

ENTR for 1 :  0.647
(IG 0.324)

ENTR for 2 :  0.551
(IG 0.420)

ENTR for 3 :  0.608
(IG 0.363)
